<a href="https://colab.research.google.com/github/fhun/Deep_learning_LSTM_playground/blob/master/6)_Stock_forecast_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pandas_datareader
from pandas_datareader import data as pdr
from tqdm import tqdm,trange
stock_list = ['KBANK','SCB','BBL','KTB']
stock_data = []
stock_name = []
for quote in tqdm(stock_list):
    try:
        stock_data.append(pdr.get_data_yahoo(f'{quote}.BK', start='2010-01-01', end='2019-04-15'))
        stock_name.append(quote)
    except:
        print("Error:", sys.exc_info()[0])
        print("Description:", sys.exc_info()[1])

100%|██████████| 4/4 [00:03<00:00,  1.18it/s]


In [0]:
stock_data[0].head()

High    Low   Open  Close      Volume  Adj Close
Date                                                         
2010-01-04  85.50  84.00  85.25  84.50   2410100.0  69.404213
2010-01-05  85.25  84.25  85.00  84.50   4058600.0  69.404213
2010-01-06  86.00  84.75  85.00  85.75   9900800.0  70.430893
2010-01-07  87.75  86.25  86.50  86.75  13073500.0  71.252266
2010-01-08  88.25  86.75  87.50  88.00  10187600.0  72.278961

In [0]:
from matplotlib import pyplot
import pandas as pd
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999
 
# function สำหรับ preprocess ข้อมูล time series หลายตัวแปร
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True,feat_name=None):
    
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [f'{feat_name[j]}(t-{i})' for j in range(n_vars)]
    
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [f'{feat_name[j]}(t)' for j in range(n_vars)]
        else:
            names += [f'{feat_name[j]}(t+{i})' for j in range(n_vars)]
    
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

# เลือกข้อมูลหุ้นจาก list ของ DataFrame และ drop column 'Close' เนื่องจากเราจะใช้ column 'Adj. Close' เท่านั้น
stock_data[0] = stock_data[0].drop('Close',axis=1)
dataset = stock_data[0]
values = dataset.values
values = values.astype('float32')

# ทำ scaling ข้อมูลด้วยวิธี min max scaling เปลี่ยน scale ข้อมูลแต่ละ column ให้อยู่ระหว่าง [0,1] และเก็บค่า min max แต่ละ column ไว้สำหรับทำ rescale ข้อมูลภายหลัง
min_dict = dict()
max_dict = dict()
for col in dataset.columns:
  min_dict[col] = dataset[col].min()
  max_dict[col] = dataset[col].max()
  dataset[col] = (dataset[col] - dataset[col].min())/(dataset[col].max()-dataset[col].min())
  
# ใช้ function สำหรับ preprocess ข้อมูลที่เขียนไว้ และ drop column ที่ไม่ได้ใช้
reframed = series_to_supervised(dataset.values, 30, 3,feat_name=stock_data[0].columns)
reframed.drop(['High(t)','High(t+1)','High(t+2)','Low(t)','Low(t+1)','Low(t+2)','Open(t)','Open(t+1)','Open(t+2)','Volume(t)','Volume(t+1)',\
               'Volume(t+2)'],axis=1,inplace=True)

In [0]:
reframed.head()

High(t-30)  Low(t-30)  Open(t-30)  Volume(t-30)  Adj Close(t-30)  \
30    0.036179   0.039301    0.043290      0.055656         0.032814   
31    0.034732   0.040757    0.041847      0.093725         0.032814   
32    0.039074   0.043668    0.041847      0.228637         0.038891   
33    0.049204   0.052402    0.050505      0.301904         0.043752   
34    0.052098   0.055313    0.056277      0.235260         0.049829   

    High(t-29)  Low(t-29)  Open(t-29)  Volume(t-29)  Adj Close(t-29)  \
30    0.034732   0.040757    0.041847      0.093725         0.032814   
31    0.039074   0.043668    0.041847      0.228637         0.038891   
32    0.049204   0.052402    0.050505      0.301904         0.043752   
33    0.052098   0.055313    0.056277      0.235260         0.049829   
34    0.053546   0.058224    0.062049      0.122073         0.047398   

    High(t-28)  Low(t-28)  Open(t-28)  Volume(t-28)  Adj Close(t-28)  \
30    0.039074   0.043668    0.041847      0.228637         0.038891   
31    0.049204   0.052402    0.050505      0.301904         0.043752   
32    0.052098   0.055313    0.056277      0.235260         0.049829   
33    0.053546   0.058224    0.062049      0.122073         0.047398   
34    0.056440   0.058224    0.054834      0.245366         0.049829   

    High(t-27)  Low(t-27)  Open(t-27)  Volume(t-27)  Adj Close(t-27)  \
30    0.049204   0.052402    0.050505      0.301904         0.043752   
31    0.052098   0.055313    0.056277      0.235260         0.049829   
32    0.053546   0.058224    0.062049      0.122073         0.047398   
33    0.056440   0.058224    0.054834      0.245366         0.049829   
34    0.050651   0.058224    0.056277      0.124724         0.049829   

    High(t-26)  Low(t-26)  Open(t-26)  Volume(t-26)  Adj Close(t-26)  \
30    0.052098   0.055313    0.056277      0.235260         0.049829   
31    0.053546   0.058224    0.062049      0.122073         0.047398   
32    0.056440   0.058224    0.054834      0.245366         0.049829   
33    0.050651   0.058224    0.056277      0.124724         0.049829   
34    0.053546   0.061135    0.062049      0.120886         0.048614   

    High(t-25)  Low(t-25)  Open(t-25)  Volume(t-25)  Adj Close(t-25)  \
30    0.053546   0.058224    0.062049      0.122073         0.047398   
31    0.056440   0.058224    0.054834      0.245366         0.049829   
32    0.050651   0.058224    0.056277      0.124724         0.049829   
33    0.053546   0.061135    0.062049      0.120886         0.048614   
34    0.052098   0.052402    0.059163      0.141857         0.042537   

    High(t-24)  Low(t-24)  Open(t-24)  Volume(t-24)  Adj Close(t-24)  \
30    0.056440   0.058224    0.054834      0.245366         0.049829   
31    0.050651   0.058224    0.056277      0.124724         0.049829   
32    0.053546   0.061135    0.062049      0.120886         0.048614   
33    0.052098   0.052402    0.059163      0.141857         0.042537   
34    0.040521   0.048035    0.047619      0.106241         0.040106   

    High(t-23)  Low(t-23)  Open(t-23)  Volume(t-23)  Adj Close(t-23)  \
30    0.050651   0.058224    0.056277      0.124724         0.049829   
31    0.053546   0.061135    0.062049      0.120886         0.048614   
32    0.052098   0.052402    0.059163      0.141857         0.042537   
33    0.040521   0.048035    0.047619      0.106241         0.040106   
34    0.041968   0.043668    0.047619      0.125207         0.034029   

    High(t-22)  Low(t-22)  Open(t-22)  Volume(t-22)  Adj Close(t-22)  \
30    0.053546   0.061135    0.062049      0.120886         0.048614   
31    0.052098   0.052402    0.059163      0.141857         0.042537   
32    0.040521   0.048035    0.047619      0.106241         0.040106   
33    0.041968   0.043668    0.047619      0.125207         0.034029   
34    0.033285   0.029112    0.040404      0.173910         0.030384   

    High(t-21)  Low(t-21)  Open(t-21)  Volume(t-21)  Adj Close(t-21)  \
30    0.052098   0.052402    0.0591

In [0]:
# split into train and test sets
values = reframed.values
n_train_percent = 0.9
split = int(values.shape[0]*0.9)
train = values[:split, :]
test = values[split:, :]

In [0]:
date_reframed = dataset.index[reframed.index]
date_train = date_reframed[:split]
date_test = date_reframed[split:]

In [0]:
# split into input and outputs
train_X, train_y = train[:, :-3], train[:, -3:]
test_X, test_y = test[:, :-3], test[:, -3:]
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

In [0]:
train_X

array([[[0.03617945, 0.03930131, 0.04329004, ..., 0.01154401,
         0.11398155, 0.00972262]],

       [[0.03473227, 0.04075691, 0.04184704, ..., 0.01010101,
         0.08878035, 0.        ]],

       [[0.03907381, 0.04366812, 0.04184704, ..., 0.        ,
         0.07224589, 0.00607674]],

       ...,

       [[0.82633864, 0.82532751, 0.82106782, ..., 0.65079365,
         0.08531643, 0.73507009]],

       [[0.83791606, 0.83114993, 0.82683983, ..., 0.65945166,
         0.08957013, 0.75246126]],

       [[0.82054993, 0.7845706 , 0.82106782, ..., 0.67965368,
         0.11830222, 0.75246126]]])

In [0]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

# design network
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(3))
model.compile(loss='mae', optimizer='adam')
# fit network
history = model.fit(train_X, train_y, epochs=100, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 2012 samples, validate on 224 samples
Epoch 1/100
 - 1s - loss: 0.0721 - val_loss: 0.0433
Epoch 2/100
 - 0s - loss: 0.0832 - val_loss: 0.0496
Epoch 3/100
 - 0s - loss: 0.0518 - val_loss: 0.0561
Epoch 4/100
 - 0s - loss: 0.0625 - val_loss: 0.0624
Epoch 5/100
 - 0s - loss: 0.0744 - val_loss: 0.0429
Epoch 6/100
 - 0s - loss: 0.0767 - val_loss: 0.0351
Epoch 7/100
 - 0s - loss: 0.0541 - val_loss: 0.0500
Epoch 8/100
 - 0s - loss: 0.0618 - val_loss: 0.0340
Epoch 9/100
 - 0s - loss: 0.0433 - val_loss: 0.0330
Epoch 10/100
 - 0s - loss: 0.0537 - val_loss: 0.0381
Epoch 11/100
 - 0s - loss: 0.0436 - val_loss: 0.0370
Epoch 12/100
 - 0s - loss: 0.0595 - val_loss: 0.0353
Epoch 13/100
 - 0s - loss: 0.0403 - val_loss: 0.0323
Epoch 14/100
 - 0s - loss: 0.0533 - val_loss: 0.0402
Epoch 15/100
 - 0s - loss: 0.0395 - val_loss: 0.0321
Epoch 16/100
 - 0s - loss: 0.0566 - val_loss: 0

In [0]:
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [0]:
# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))

In [0]:
inv_yhat = yhat*(max_dict['Adj Close']-min_dict['Adj Close'])+min_dict['Adj Close']
inv_y = test_y*(max_dict['Adj Close']-min_dict['Adj Close'])+min_dict['Adj Close']

In [0]:
import numpy as np
# also show the price of previous 30 days + datetime
idx = np.random.randint(0,len(yhat-3))
if idx <30:
  date_actual = date_test[:idx+3]
else:
  date_actual = date_test[idx-30:idx+3]

_=pyplot.plot(date_actual,inv_y[max(0,idx-30):idx+3,0],'b')
_=pyplot.plot(date_test[idx:idx+3],inv_yhat[idx],'r')